In [58]:
from tqdm import tqdm

In [59]:
set_seed(42)

In [60]:
def set_device(device_no: int):
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("There are %d GPU(s) available." % torch.cuda.device_count())
        print("We will use the GPU:", torch.cuda.get_device_name(device_no))
    else:
        print("No GPU available, using the CPU instead.")
        device = torch.device("cpu")
        
    return device

device = set_device(4)

There are 8 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


## setup

In [207]:
from configs import *
import clip
import open_clip
from open_clip import tokenizer
from PIL import Image
from sklearn.manifold import TSNE
from bokeh.io import output_notebook
from sklearn.decomposition import PCA
import bokeh.models as bm, bokeh.plotting as pl
from sklearn.preprocessing import StandardScaler


def get_image_features(path_to_im: str, classes: dict, N: int, model_name="ViT-B-16", model_author="openai",
                       device="cuda"):
    """
    Args:
        path_to_im: path to the directory with images data
        classes: dict of class idx, class label pairs
        N: how many images of every class to take
        model_name: name of CLIP model's Image Encoder
        model_author: pretrained name of CLIP model from open_clip
        device: device
    Return: torch.Tensor of encoded by CLIP model images
    """
    image_dirs = [path_to_im + "/{}/".format(cls) for cls in classes.values()]

    selected_images = []
    for direc in image_dirs:
        filenames = os.listdir(direc)
        upd_filenames = [direc + f for f in filenames]
        selected_images.append(sorted(upd_filenames[:N]))

    selected_images = [item for sublist in selected_images for item in sublist]

    image_inputs = []
    clip_model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=model_author,
                                                                      device=device)
    for name in tqdm(selected_images):
        im = Image.open(name)
        im_input = preprocess(im).unsqueeze(0).to(device)
        image_inputs.append(im_input)

    image_inputs = torch.stack(image_inputs, dim=0)

    image_encodings = []
    with torch.no_grad():
        for image in tqdm(image_inputs):
            image_feature = clip_model.encode_image(image)
            image_encodings.append(image_feature)

    image_features = torch.stack(image_encodings, dim=0)

    return image_features

def remove_prefixes(strings):
    prefixes = ['a', 'an', 'the']
    result = []
    for string in strings:
        words = string.split()
        if words[0].lower() in prefixes:
            result.append(' '.join(words[1:]))
        else:
            result.append(string)

    return result


def get_text_features(path_to_text: str, model_name="ViT-B-16", model_author="openai", device="cuda"):
    """
    Args:
        path_to_text: path to the directory with texts data
        model_name: name of CLIP model's Image Encoder
        model_author: pretrained name of CLIP model from open_clip
        device: device
    Return: torch.Tensor of encoded by CLIP model texts
    """
    with open(path_to_text, "r") as f:
        texts = f.read().lower().split("\n")
        texts = remove_prefixes(texts)

    text_encodings = []
    clip_model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=model_author,
                                                                      device=device)

    with torch.no_grad():
        for c in tqdm(texts):
            text_input = clip.tokenize(c).to(device)
            text_feature = clip_model.encode_text(text_input)
            text_encodings.append(text_feature)

    text_features = torch.stack(text_encodings, dim=0)

    return text_features
def similarity(a: torch.Tensor, b: torch.Tensor):
    nom = a @ b.T
    denom = a.norm(dim=-1) * b.norm(dim=-1)
    return nom / denom

def cubed_similarity(a: torch.Tensor, b: torch.Tensor):
    nom = a**3 @ (b**3).T
    denom = (a**3).norm(dim=-1) * (b**3).norm(dim=-1)
    return nom / denom

def get_dot_prods_matrix(image_features: torch.Tensor, text_features: torch.Tensor, eps=1e-8):
    """
    Args:
        image_features: tensor of shape [num_images, dim]
        text_features: tensor of shape [num_texts, dim]
        eps: to avoid division by zero
    Return: images-texts matrix with normalized rows (sum of each row == 1.)
    """
    image_features = image_features.squeeze(dim=1)
    text_features = text_features.squeeze(dim=1)
    image_norms = image_features.norm(dim=-1, keepdim=True) ** 2
    text_norms = text_features.norm(dim=-1, keepdim=True) ** 2
    matrix = image_features @ text_features.T
    matrix /= torch.sqrt(image_norms @ text_norms.T)
    row_sum = torch.sum(matrix, dim=1, keepdim=True)
    matrix /= row_sum + eps
    return matrix

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxiliary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({'x': x, 'y': y, 'color': color, **kwargs})

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

def generate_slices_for_classes(classes: dict, V_matrix: torch.Tensor):
    """
    classes: dict of cls_idx: class_name
    V_matrix: image-concepts matrix
    return: slices for every class
    """
    slice_size = len(V_matrix) // len(classes)
    slices = {}
    for i, class_idx in enumerate(classes):
        start = i * slice_size
        end = (i + 1) * slice_size
        slices[classes[class_idx]] = slice(start, end)

    return slices

def calculate_similarity_score(classes: dict, V_rows: torch.Tensor, T_matrix: torch.Tensor, sim: str="sim"):
    """
    V_rows: a rows of V_matrix for the same class
    T_matrix: classes-concepts matrix
    return: a dictionary with similarity scores for each class
    """
    scores = {}

    for v_row in V_rows:
        for i in range(len(T_matrix)):
            t_row = T_matrix[i]

            if sim == "sim":
                sim = similarity(v_row, t_row)
            else:
                sim = cubed_similarity(v_row, t_row)

            class_name = classes[i]

            if class_name in scores:
                scores[class_name] += sim.item()
            else:
                scores[class_name] = sim.item()

    return scores

def get_scores_dict(classes: dict, V_matrix: torch.Tensor, T_matrix: torch.Tensor):
    """
    Args:
        classes: classes dict
        V_matrix:
        T_matrix:
    Return: scores_dict for drawing similarity scores
    """
    scores_dict = {}
    slices = generate_slices_for_classes(classes, V_matrix)

    for class_name, slice_range in slices.items():
        V_rows = V_matrix[slice_range]
        scores = calculate_similarity_score(classes, V_rows, T_matrix)
        scores_dict[class_name] = scores

    return scores_dict

def draw_similarity_scores(scores_dict: dict, true_class: str):
    """
    scores_dict: a nested dictionary with similarity scores
    true_class: the true image class for which scores should be plotted
    """

    if true_class not in scores_dict:
        print(f"True class '{true_class}' not found in the scores dictionary.")
        return

    scores = scores_dict[true_class]
    df = pd.DataFrame(list(scores.items()), columns=['Class', 'Total Similarity Score'])

    plt.figure(figsize=(6, 3))  # 12 6
    sns.scatterplot(data=df, x='Class', y='Total Similarity Score')
    plt.title(f"Similarity Scores for True Class: {true_class}")
    plt.xticks(rotation=45)
    plt.xlabel("Classes")
    plt.ylabel("Total Similarity Score")
    plt.tight_layout()
    plt.show()
    
def calculate_similarity_score_accuracy_for_class(class_name: str, V_matrix: torch.Tensor, T_matrix: torch.Tensor, classes: dict):
    """
    Test the accuracy of the hypothesis
    return: accuracy score for the class_name label by similarity method
    """
    slices = generate_slices_for_classes(classes, V_matrix)
    slice_range = slices[class_name]
    V_rows = V_matrix[slice_range]
    sim_matrix = torch.zeros((V_rows.shape[0], T_matrix.shape[0]))
    
    correct, total = 0, 0
    
    for i, v_row in enumerate(V_rows):
        for j, t_row in enumerate(T_matrix):
            sim_matrix[i, j] = similarity(v_row, t_row).item()
            
    for idx in range(sim_matrix.shape[0]):
        pred_idx = torch.argmax(sim_matrix[idx])
        
        if pred_idx == list(classes.values()).index(class_name):
            correct += 1.0
        total += 1.0
        
    return 100 * correct / total
    
def similarity_score_accuracy(classes: dict, V_matrix: torch.Tensor, T_matrix: torch.Tensor):
    mean = np.mean([calculate_similarity_score_accuracy_for_class(class_name, V_matrix, T_matrix, classes) for class_name in classes.values()])
    return "Similarity Score accuracy: {}%".format(mean)

def calculate_max_score_accuracy_for_class(class_name: str, V_matrix: torch.Tensor, T_matrix: torch.Tensor, classes: dict):
    """
    Args:
        class_name: name of class from classes dict
        V_matrix: yeap
        T_matrix: yeap
        classes: classes dict
    Return:
    """
    slices = generate_slices_for_classes(classes, V_matrix)
    slice_range = slices[class_name]
    V_rows = V_matrix[slice_range]

    correct, total = 0, 0

    for v_row in V_rows:
        max_elem_idx = torch.argmax(v_row).item()
        t_matrix_column = T_matrix[:, max_elem_idx]
        t_max_elem_idx = torch.argmax(t_matrix_column).item()

        if classes[t_max_elem_idx] == class_name:
            correct += 1
        total += 1

    return 100 * correct / total

def max_score_accuracy(classes: dict, V_matrix: torch.Tensor, T_matrix: torch.Tensor):
    mean = np.mean([calculate_max_score_accuracy_for_class(class_name, V_matrix, T_matrix, classes) for class_name in classes.values()])
    return "Max Score accuracy: {}%".format(mean)

## experiments

In [62]:
classes = {}

with open("./datasets/cifar10/labels.txt", "r") as f:
    cls_names = f.read().lower().split("\n")

for i, cls_name in enumerate(cls_names):
    classes[i] = cls_name


print(classes)

{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [173]:
image_features = get_image_features("./datasets/cifar10/train", classes=classes, N=50, device=device)

100%|████████████████████████████████████████| 500/500 [00:02<00:00, 170.34it/s]


In [174]:
concept_features = get_text_features("all_concepts.txt", device=device)
class_features = get_text_features("datasets/cifar10/labels.txt", device=device)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 130.23it/s]


In [175]:
V_matrix = get_dot_prods_matrix(image_features, concept_features)
T_matrix = get_dot_prods_matrix(class_features, concept_features)

print('V_matrix:', V_matrix.shape, "\n")
print('T_matrix:', T_matrix.shape)

V_matrix: torch.Size([500, 5051]) 

T_matrix: torch.Size([10, 5051])


In [213]:
print(similarity_score_accuracy(classes, V_matrix, T_matrix), "\n")
print(max_score_accuracy(classes, V_matrix, T_matrix))

Similarity Score accuracy: 61.0% 

Max Score accuracy: 68.4%


In [177]:
def get_dot_prods_matrix_test(image_features: torch.Tensor, text_features: torch.Tensor):
    image_features = image_features.squeeze(dim=1)
    text_features = text_features.squeeze(dim=1)
    image_norms = image_features.norm(dim=-1, keepdim=True) ** 2
    text_norms = text_features.norm(dim=-1, keepdim=True) ** 2
    matrix = image_features @ text_features.T
    matrix /= torch.sqrt(image_norms @ text_norms.T)
    return matrix

V_matrix_non_norm = get_dot_prods_matrix_test(image_features, concept_features)
T_matrix_non_norm = get_dot_prods_matrix_test(class_features, concept_features)

v_col_sum = torch.sum(V_matrix_non_norm, dim=0, keepdim=True)
t_col_sum = torch.sum(T_matrix_non_norm, dim=0, keepdim=True)

V_matrix_norm_col = V_matrix_non_norm / v_col_sum
T_matrix_norm_col = T_matrix_non_norm / t_col_sum

print(similarity_score_accuracy(classes, V_matrix_norm_col, T_matrix_norm_col), "\n")
print(max_score_accuracy(classes, V_matrix_norm_col, T_matrix_norm_col))

Similarity Score accuarcy: 65.6 

Max Score accuracy: 63.2%


max score accuracy is better when we normalize by rows while the similarity accuracy better when we normalize by rows

In [208]:
for class_name in classes.values():
    print("Similarity Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_similarity_score_accuracy_for_class(class_name, V_matrix, T_matrix, classes)))

Similarity Score accuracy for the class airplane 80.0%
Similarity Score accuracy for the class automobile 0.0%
Similarity Score accuracy for the class bird 88.0%
Similarity Score accuracy for the class cat 0.0%
Similarity Score accuracy for the class deer 92.0%
Similarity Score accuracy for the class dog 0.0%
Similarity Score accuracy for the class frog 72.0%
Similarity Score accuracy for the class horse 92.0%
Similarity Score accuracy for the class ship 86.0%
Similarity Score accuracy for the class truck 100.0%


In [210]:
for class_name in classes.values():
    print("Similarity Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_similarity_score_accuracy_for_class(class_name, V_matrix_norm_col, T_matrix_norm_col, classes)))

Similarity Score accuracy for the class airplane 66.0%
Similarity Score accuracy for the class automobile 20.0%
Similarity Score accuracy for the class bird 80.0%
Similarity Score accuracy for the class cat 24.0%
Similarity Score accuracy for the class deer 86.0%
Similarity Score accuracy for the class dog 26.0%
Similarity Score accuracy for the class frog 76.0%
Similarity Score accuracy for the class horse 98.0%
Similarity Score accuracy for the class ship 92.0%
Similarity Score accuracy for the class truck 88.0%


In [178]:
for class_name in classes.values():
    print("Max Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_max_score_accuracy_for_class(class_name, V_matrix, T_matrix, classes)))

Max Score accuracy for the class airplane 72.0%
Max Score accuracy for the class automobile 78.0%
Max Score accuracy for the class bird 78.0%
Max Score accuracy for the class cat 54.0%
Max Score accuracy for the class deer 84.0%
Max Score accuracy for the class dog 58.0%
Max Score accuracy for the class frog 50.0%
Max Score accuracy for the class horse 82.0%
Max Score accuracy for the class ship 66.0%
Max Score accuracy for the class truck 62.0%


In [171]:
for class_name in classes.values():
    print("Max Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_max_score_accuracy_for_class(class_name, V_matrix_norm_col, T_matrix_norm_col, classes)))

Max Score accuracy for the class airplane 36.0%
Max Score accuracy for the class automobile 32.0%
Max Score accuracy for the class bird 74.0%
Max Score accuracy for the class cat 64.0%
Max Score accuracy for the class deer 38.0%
Max Score accuracy for the class dog 74.0%
Max Score accuracy for the class frog 70.0%
Max Score accuracy for the class horse 92.0%
Max Score accuracy for the class ship 80.0%
Max Score accuracy for the class truck 72.0%


# json llama concetpts tests

In [98]:
import json

with open("concepts_llama_mixed_prompts_chunck2.json", "r") as fp:
    llama_concepts_raw = json.load(fp)

In [201]:
import re

llama_concepts = []

for key, value in llama_concepts_raw.items():
    if "A1" in value:
        a1_text = value["A1"]
        lines = a1_text.split("\n")
        lines.pop(0)
        lines = [element for element in lines if element != '']
        lines = [element for element in lines if not element.strip().replace(".", "").isdigit()]
        lines = [re.sub(r'[0-9.]', '', line) for line in lines]
        lines = [re.sub(r'[*|-\–_+()]', '', line) for line in lines]
        lines = [re.sub(r'[---]', '', line) for line in lines]
        lines = [re.sub(r'[A-J\t]', '', line) for line in lines]
        lines = [re.sub(r'[#]', '', line) for line in lines]
        lines = [element for element in lines if element != '']
        lines = [line.lstrip() for line in lines]
        for line in lines:
            if len(line) <= 25:
                llama_concepts.append(line)
                    
    if "A2" in value:
        a2_text = value["A2"]
        lines = a2_text.split('\n')
        lines.pop(0)
        lines = [element for element in lines if element != '']
        lines = [element for element in lines if not element.strip().replace(".", "").isdigit()]
        lines = [re.sub(r'[0-9.]', '', line) for line in lines]
        lines = [re.sub(r'[*|-\–_+()]', '', line) for line in lines]
        lines = [re.sub(r'[---]', '', line) for line in lines]
        lines = [re.sub(r'[A-J\t]', '', line) for line in lines]
        lines = [re.sub(r'[#]', '', line) for line in lines]
        lines = [element for element in lines if element != '']
        lines = [line.lstrip() for line in lines]
        for line in lines:
            if len(line) <= 25:
                llama_concepts.append(line)  

In [204]:
with open("llama_concepts_test.txt", "w") as f:
    for i, item in enumerate(llama_concepts):
        if i < len(llama_concepts) - 1:
            f.write(item.lower() + "\n")
        else: f.write(item)

In [138]:
llama_concept_features = get_text_features("llama_concepts_test.txt", device=device)

100%|██████████████████████████████████████| 1196/1196 [00:09<00:00, 127.80it/s]


In [139]:
V_matrix_llama = get_dot_prods_matrix(image_features, llama_concept_features)
T_matrix_llama = get_dot_prods_matrix(class_features, llama_concept_features)

print('V_matrix:', V_matrix_llama.shape, "\n")
print('T_matrix:', T_matrix_llama.shape)

V_matrix: torch.Size([500, 1196]) 

T_matrix: torch.Size([10, 1196])


In [140]:
print(similarity_score_accuracy(classes, V_matrix_llama, T_matrix_llama), "\n")
print(max_score_accuracy(classes, V_matrix_llama, T_matrix_llama))

Similarity Score accuarcy: 57.6 

Max Score accuracy: 55.6%


In [167]:
V_matrix_llama_non_norm = get_dot_prods_matrix_test(image_features, llama_concept_features)
T_matrix_llama_non_norm = get_dot_prods_matrix_test(class_features, llama_concept_features)

v_col_sum = torch.sum(V_matrix_llama_non_norm, dim=0, keepdim=True)
t_col_sum = torch.sum(T_matrix_llama_non_norm, dim=0, keepdim=True)

V_matrix_llama_norm_col = V_matrix_llama_non_norm / v_col_sum
T_matrix_llama_norm_col = T_matrix_llama_non_norm / t_col_sum

print(similarity_score_accuracy(classes, V_matrix_llama_norm_col, T_matrix_llama_norm_col), "\n")
print(max_score_accuracy(classes, V_matrix_llama_norm_col, T_matrix_llama_norm_col))

Similarity Score accuarcy: 69.0 

Max Score accuracy: 59.4%


In [168]:
for class_name in classes.values():
    print("Max Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_max_score_accuracy_for_class(class_name, V_matrix_llama, T_matrix_llama, classes)))

Max Score accuracy for the class airplane 84.0%
Max Score accuracy for the class automobile 90.0%
Max Score accuracy for the class bird 76.0%
Max Score accuracy for the class cat 10.0%
Max Score accuracy for the class deer 48.0%
Max Score accuracy for the class dog 12.0%
Max Score accuracy for the class frog 58.0%
Max Score accuracy for the class horse 58.0%
Max Score accuracy for the class ship 40.0%
Max Score accuracy for the class truck 80.0%


In [169]:
for class_name in classes.values():
    print("Max Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_max_score_accuracy_for_class(class_name, V_matrix_llama_norm_col, T_matrix_llama_norm_col, classes)))

Max Score accuracy for the class airplane 0.0%
Max Score accuracy for the class automobile 54.0%
Max Score accuracy for the class bird 92.0%
Max Score accuracy for the class cat 12.0%
Max Score accuracy for the class deer 56.0%
Max Score accuracy for the class dog 58.0%
Max Score accuracy for the class frog 88.0%
Max Score accuracy for the class horse 88.0%
Max Score accuracy for the class ship 76.0%
Max Score accuracy for the class truck 70.0%


# another set

https://relatedwords.org

In [143]:
custom_concepts = ['plane', 'airliner', 'propeller', 'monoplane', 'fuselage', 'jet', 'car', 'vehicle', 'passenger', 'internal combustion engine', 'minivan', 'sedan',
                  'parrot', 'wing', 'passerine', 'fowl', 'albatross', 'geese', 'kiwi', 'syrinx', 'gull', 'lion', 'tiger', 'leopard', 'pet', 'jaguar', 'felis', 'rat', 'rabbit',
                  'elk', 'moose', 'reindeer', 'antelope', 'ruminant', 'antler', 'pig', 'sheep', 'alps', 'eurasian elk', 'foxes', 'puppy', 'cur', 'wolf', 'tail', 'great dane', 'poodle', 'hound',
                  'canid', 'corgi', 'pawl', 'toad', 'amphibian', 'egg', 'gill', 'lizard', 'tongue', 'carnivore', 'pony', 'foal', 'thoroughbred', 'hack', 'cartilage', 'donkey', 'bridle', 'boat',
                  'ferry', 'submarine', 'vessel', 'cargo', 'sail', 'sea', 'barque', 'schooner', 'travel', 'galley', 'water', 'watercraft', 'lorry', 'van', 'wagon', 'suv', 'jeep', 'forklift', 
                  'bumper', 'trailer', 'driver', 'boxcar', 'flatcar', 'motorbikr', 'oldsmobile']

In [145]:
with open("custom_concepts_test.txt", "w") as f:
    for item in custom_concepts:
        f.write(item + "\n")

In [150]:
import open_clip
import clip
from open_clip import tokenizer

clip_model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='openai', device=device)

clip_model.eval()
context_length = clip_model.context_length
vocab_size = clip_model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 149,620,737
Context length: 77
Vocab size: 49408


In [152]:
#custom_concepts_features = get_text_features("custom_concepts_test.txt", device=device)
text_encodings = []
with torch.no_grad():
        for c in tqdm(custom_concepts):
            text_input = clip.tokenize(c).to(device)
            text_feature = clip_model.encode_text(text_input)
            text_encodings.append(text_feature)

custom_concepts_features = torch.stack(text_encodings, dim=0)

V_matrix_custom = get_dot_prods_matrix(image_features, custom_concepts_features)
T_matrix_custom = get_dot_prods_matrix(class_features, custom_concepts_features)

print('V_matrix:', V_matrix_custom.shape, "\n")
print('T_matrix:', T_matrix_custom.shape)

100%|██████████████████████████████████████████| 90/90 [00:00<00:00, 123.48it/s]

V_matrix: torch.Size([500, 90]) 

T_matrix: torch.Size([10, 90])


In [212]:
print(similarity_score_accuracy(classes, V_matrix_custom, T_matrix_custom), "\n")
print(max_score_accuracy(classes, V_matrix_custom, T_matrix_custom))

Similarity Score accuracy: 67.4% 

Max Score accuracy: 68.2%


In [165]:
V_matrix_custom_non_norm = get_dot_prods_matrix_test(image_features, custom_concepts_features)
T_matrix_custom_non_norm = get_dot_prods_matrix_test(class_features, custom_concepts_features)

v_col_sum = torch.sum(V_matrix_custom_non_norm, dim=0, keepdim=True)
t_col_sum = torch.sum(T_matrix_custom_non_norm, dim=0, keepdim=True)

V_matrix_custom_norm_col = V_matrix_custom_non_norm / v_col_sum
T_matrix_custom_norm_col = T_matrix_custom_non_norm / t_col_sum

print(similarity_score_accuracy(classes, V_matrix_custom_norm_col, T_matrix_custom_norm_col), "\n")
print(max_score_accuracy(classes, V_matrix_custom_norm_col, T_matrix_custom_norm_col))

Similarity Score accuarcy: 85.0 

Max Score accuracy: 73.8%


In [163]:
for class_name in classes.values():
    print("Max Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_max_score_accuracy_for_class(class_name, V_matrix_custom, T_matrix_custom, classes)))

Max Score accuracy for the class airplane 92.0%
Max Score accuracy for the class automobile 90.0%
Max Score accuracy for the class bird 70.0%
Max Score accuracy for the class cat 6.0%
Max Score accuracy for the class deer 92.0%
Max Score accuracy for the class dog 22.0%
Max Score accuracy for the class frog 48.0%
Max Score accuracy for the class horse 82.0%
Max Score accuracy for the class ship 92.0%
Max Score accuracy for the class truck 88.0%


In [166]:
for class_name in classes.values():
    print("Max Score accuracy for the class {}".format(class_name), 
          "{}%".format(calculate_max_score_accuracy_for_class(class_name, V_matrix_custom_norm_col, T_matrix_custom_norm_col, classes)))

Max Score accuracy for the class airplane 86.0%
Max Score accuracy for the class automobile 88.0%
Max Score accuracy for the class bird 84.0%
Max Score accuracy for the class cat 20.0%
Max Score accuracy for the class deer 86.0%
Max Score accuracy for the class dog 58.0%
Max Score accuracy for the class frog 58.0%
Max Score accuracy for the class horse 92.0%
Max Score accuracy for the class ship 68.0%
Max Score accuracy for the class truck 98.0%
